### 파이썬 버전 확인

In [3]:
import sys
print(sys.version)

3.11.14 (main, Oct 21 2025, 18:27:30) [Clang 20.1.8 ]


## 네이버 쇼핑 리뷰를 보고 판매자 답변 자동 생성하기(정중/사과/보상 제안)

#### 데이터 출처 : https://github.com/bab2min/corpus

1. 라이브러리 설치 (Colab GPU / Python 3.11)

In [ ]:
# Colab은 보통 torch 2.6.0+cu124가 이미 설치되어 있습니다.
# 설치가 안 되어 있거나 버전을 맞추고 싶다면 주석 해제하여 실행하세요.
# !pip install -q --index-url https://download.pytorch.org/whl/cu124 \
#     torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124

!pip install -q -U pip
# sklearn-compat 경고 해결: scikit-learn 1.6.x로 고정
!pip install -q -U "transformers>=4.41" "datasets>=2.19" accelerate evaluate \
"scikit-learn>=1.2,<1.7" tqdm peft

2. 데이터 파싱

In [4]:
from pathlib import Path
import random

RAW_PATH = Path("./naver_shopping.txt")

def load_reviews(min_len=5, max_n=None, seed=42):
    random.seed(seed)
    rows = []
    with RAW_PATH.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t", 1)
            if len(parts) != 2:
                continue
            rating_s, text = parts
            try:
                rating = int(rating_s)
            except:
                continue
            text = text.strip()
            if len(text) < min_len:
                continue
            rows.append((rating, text))
    random.shuffle(rows)
    if max_n:
        rows = rows[:max_n]
    return rows

rows = load_reviews(max_n=30000)  # 8GB면 처음엔 3만 이하 추천
print("loaded:", len(rows))
print(rows[0])

loaded: 30000
(2, '냄새랑 맛이 생각보다 너무 역해요')


3. 불만 유형(키워드) 분류

In [5]:
COMPLAINT_RULES = {
    "배송": ["배송", "늦", "지연", "도착", "택배"],
    "품질/불량": ["불량", "하자", "고장", "깨", "찢", "누수", "작동", "불안정"],
    "포장": ["포장", "박스", "파손", "훼손", "완충"],
    "가격/가성비": ["비싸", "가격", "가성비", "싸", "할인"],
    "응대/서비스": ["응대", "문의", "연락", "고객센터", "불친절"],
}

def detect_topics(text: str):
    topics = []
    for topic, kws in COMPLAINT_RULES.items():
        if any(kw in text for kw in kws):
            topics.append(topic)
    return topics[:2]  # 너무 많으면 1~2개만

4. 템플릿 기반 판매자 답변 생성

In [6]:
def make_seller_reply(rating: int, review: str) -> str:
    topics = detect_topics(review)
    topic_phrase = " / ".join(topics) if topics else None

    # 보상 제안(너무 구체적 금액은 피하고 옵션만 제시)
    compensation = random.choice([
        "교환 또는 환불을 도와드리겠습니다.",
        "확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.",
        "불편을 줄이기 위해 교환/환불 절차를 빠르게 진행해드리겠습니다."
    ])

    # 긍정(4~5)
    if rating >= 4:
        extra = "앞으로도 더 좋은 상품과 서비스로 보답하겠습니다."
        upsell = random.choice([
            "재구매해주시면 감사하겠습니다.",
            "다음에도 만족스러운 경험을 드리겠습니다.",
            "소중한 후기 감사합니다."
        ])
        if topic_phrase:
            return f"안녕하세요, 고객님. {topic_phrase} 관련하여 만족하셨다니 정말 기쁩니다. {extra} {upsell}"
        return f"안녕하세요, 고객님. 소중한 후기 감사합니다. {extra} {upsell}"

    # 부정(1~2)
    if rating <= 2:
        apology = "불편을 드려 진심으로 죄송합니다."
        ask = "주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다."
        if topic_phrase:
            return f"안녕하세요, 고객님. {topic_phrase} 관련하여 {apology} {ask} {compensation}"
        return f"안녕하세요, 고객님. {apology} {ask} {compensation}"

    # 중립(3) — 과제에서는 제외해도 되고, 남겨도 됨
    neutral = "의견 남겨주셔서 감사합니다."
    improve = "말씀해주신 부분은 개선하여 더 나은 서비스로 보답하겠습니다."
    if topic_phrase:
        return f"안녕하세요, 고객님. {topic_phrase} 관련하여 {neutral} {improve}"
    return f"안녕하세요, 고객님. {neutral} {improve}"

# 샘플 확인
for r, t in rows[:5]:
    print("RATING:", r)
    print("REVIEW:", t)
    print("REPLY:", make_seller_reply(r, t))
    print("-"*80)

RATING: 2
REVIEW: 냄새랑 맛이 생각보다 너무 역해요
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.
--------------------------------------------------------------------------------
RATING: 5
REVIEW: 진짜대박좋아요만족^^
REPLY: 안녕하세요, 고객님. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 다음에도 만족스러운 경험을 드리겠습니다.
--------------------------------------------------------------------------------
RATING: 5
REVIEW: 좋아요 배송도 빠르고 가겨도 마니 저렴하고~~^^
REPLY: 안녕하세요, 고객님. 배송 관련하여 만족하셨다니 정말 기쁩니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 다음에도 만족스러운 경험을 드리겠습니다.
--------------------------------------------------------------------------------
RATING: 1
REVIEW: 1년 넘게 쓴 와이퍼보다 능력이 떨어지내요. 워셔액도 못 닦아내네요. 이런거 팔지 않으셨으면 좋겠네요
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.
--------------------------------------------------------------------------------
RATING: 2
REVIEW: 진짜뚱뚱하신분만사셔야될듯
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문

5. 학습 데이터 구성 + split

In [7]:
from sklearn.model_selection import train_test_split

def build_example(rating, review):
    # 프롬프트: 역할/규칙/입력
    prompt = (
        "### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n"
        "### 규칙:\n"
        "- 한국어 존댓말로 정중하게 작성\n"
        "- 2~4문장으로 간결하게\n"
        "- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함\n"
        "- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함\n"
        "### 고객 리뷰:\n"
        f"{review}\n"
        "### 판매자 답변:\n"
    )
    completion = make_seller_reply(rating, review)
    return {"prompt": prompt, "completion": completion, "rating": rating, "review": review}

# 3점은 빼서 “명확한” 긍/부정만 학습(추천)
filtered = [(r, t) for (r, t) in rows if r in (1,2,4,5)]
data = [build_example(r, t) for r, t in filtered[:20000]]  # 최대 2만개만

train_data, valid_data = train_test_split(data, test_size=0.05, random_state=42)

print(len(train_data), len(valid_data))
print(train_data[0]["prompt"])
print("->", train_data[0]["completion"])

19000 1000
### 역할: 당신은 온라인 쇼핑몰 판매자입니다.
### 규칙:
- 한국어 존댓말로 정중하게 작성
- 2~4문장으로 간결하게
- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함
- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함
### 고객 리뷰:
저렴하게 좋은 제품을 구매한거 같아 만족해요^^
### 판매자 답변:

-> 안녕하세요, 고객님. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 재구매해주시면 감사하겠습니다.


6. 모델/토크나이저 로드 + LoRA 적용

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

MODEL_ID = "skt/kogpt2-base-v2"   # 안 되면 여기만 다른 KoGPT2 repo로 변경
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# GPT2류는 pad_token이 없는 경우가 많아서 eos로 맞추는 게 안전
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
model.config.use_cache = False
model.gradient_checkpointing_enable()  # 메모리 절약

# LoRA 설정 (GPT-2 계열에서 흔히 쓰는 target)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj"],  # GPT-2 계열에 흔한 모듈명
    fan_in_fan_out=True
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.to(device)

device: cuda


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

trainable params: 811,008 || all params: 125,975,040 || trainable%: 0.6438


model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(51200, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

7. Dataset 만들기 + 토크나이징

In [13]:
from datasets import Dataset

def pack_text(ex):
    return ex["prompt"] + ex["completion"]

train_ds = Dataset.from_dict({"text": [pack_text(x) for x in train_data]})
valid_ds = Dataset.from_dict({"text": [pack_text(x) for x in valid_data]})

MAX_LEN = 192  # 8GB 추천(256도 가능하지만 터질 수 있음)

def tok(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length"
    )

train_tok = train_ds.map(tok, batched=True, remove_columns=["text"])
valid_tok = valid_ds.map(tok, batched=True, remove_columns=["text"])


Map:   0%|          | 0/19000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

8. Trainer로 학습

In [16]:
import os, random, inspect
import torch
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)

from peft import get_peft_model, LoraConfig, TaskType

# ----- 전제 체크 -----
assert "tokenizer" in globals(), "tokenizer가 먼저 정의돼야 합니다."
assert "train_tok" in globals(), "train_tok(토큰화된 학습 데이터셋)이 먼저 정의돼야 합니다."
valid_tok = globals().get("valid_tok", None)
MODEL_ID = globals().get("MODEL_ID", "skt/kogpt2-base-v2")

print("torch =", torch.__version__)
print("cuda available =", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu name =", torch.cuda.get_device_name(0))

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device =", device)

# ----- (1) tokenizer / special token 안전 확인 -----
# pad_token_id가 없으면 eos를 pad로 임시 사용(이미 세팅돼 있으면 건드리지 않음)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

print("len(tokenizer) =", len(tokenizer))
print("pad_token_id =", tokenizer.pad_token_id, "eos_token_id =", tokenizer.eos_token_id)

# ----- (2) 베이스 모델 로드 + ✅ 임베딩 리사이즈(너 케이스의 핵심) -----
base_model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

# 51200 -> 51201로 맞추는 부분 (IndexError 해결)
base_model.resize_token_embeddings(len(tokenizer))

base_model.config.pad_token_id = tokenizer.pad_token_id
if tokenizer.eos_token_id is not None:
    base_model.config.eos_token_id = tokenizer.eos_token_id

# 메모리 절약(특히 GPU에서 유용)
base_model.config.use_cache = False
base_model.gradient_checkpointing_enable()

# ----- (3) LoRA 설정 -----
lora_config = globals().get("lora_config", None)
if lora_config is None:
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        target_modules=["c_attn"],  # KoGPT2/GPT2에서 흔히 존재
    )

model = get_peft_model(base_model, lora_config)
model.to(device)

try:
    model.print_trainable_parameters()
except Exception:
    pass

# ----- (4) 데이터 콜레이터 -----
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ----- (5) TrainingArguments (버전 호환 처리 포함) -----
OUTPUT_DIR = "outputs/seller_reply_lora"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# transformers 버전에 따라 eval_strategy / evaluation_strategy 이름이 다를 수 있어 try/except
common_kwargs = dict(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    max_steps=200,
    num_train_epochs=1,
    logging_steps=25,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,  # ✅ PEFT에서 labels 컬럼 삭제 방지 (중요)
    fp16=torch.cuda.is_available(),  # GPU면 fp16
    seed=42,
)

try:
    # 최신 버전 계열
    args = TrainingArguments(
        **common_kwargs,
        eval_strategy="steps" if valid_tok is not None else "no",
        eval_steps=200 if valid_tok is not None else None,
    )
except TypeError:
    # 구버전 계열
    args = TrainingArguments(
        **common_kwargs,
        evaluation_strategy="steps" if valid_tok is not None else "no",
        eval_steps=200 if valid_tok is not None else None,
    )

# ----- (6) ✅ Trainer 생성 (중요: label_names는 여기 넣지 말 것) -----
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=valid_tok if valid_tok is not None else None,
    data_collator=collator,
)

# (선택) Trainer에 label_names 속성이 있으면 설정해서 경고 줄이기(버전별 호환)
if hasattr(trainer, "label_names"):
    trainer.label_names = ["labels"]

# ----- (7) 마지막 범위 체크 (너의 out-of-range 재발 방지) -----
emb_size = model.get_input_embeddings().weight.shape[0]
idxs = random.sample(range(len(train_tok)), k=min(200, len(train_tok)))
mx = max(max(train_tok[i]["input_ids"]) for i in idxs)
print("model emb size =", emb_size, "| sample input_id max =", mx)
assert mx < emb_size, f"토큰 id({mx})가 임베딩 크기({emb_size}) 이상입니다. resize가 적용됐는지 확인하세요."

# ----- (8) 학습 -----
train_result = trainer.train()

# ----- (9) 저장 -----
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Done. Saved to:", OUTPUT_DIR)
print("Train result:", train_result)


torch = 2.6.0+cu124
cuda available = True
gpu name = Tesla T4
device = cuda
len(tokenizer) = 51201
pad_token_id = 51200 eos_token_id = 51200


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 811,008 || all params: 125,975,808 || trainable%: 0.6438
model emb size = 51201 | sample input_id max = 51200


Step,Training Loss,Validation Loss
200,0.693200,0.657933


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


✅ Done. Saved to: outputs/seller_reply_lora
Train result: TrainOutput(global_step=200, training_loss=1.2322107982635497, metrics={'train_runtime': 316.4756, 'train_samples_per_second': 10.111, 'train_steps_per_second': 0.632, 'total_flos': 316540138291200.0, 'train_loss': 1.2322107982635497, 'epoch': 0.16842105263157894})


9. 생성 함수

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

MODEL_ID = "skt/kogpt2-base-v2"          # 셀 10에서 쓴 베이스 모델과 동일해야 함
CKPT_DIR = "outputs/seller_reply_lora"   # 셀 10에서 저장한 경로와 동일

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device =", device)

# 1) ✅ tokenizer를 '저장 폴더'에서 로드 (len=51201 유지)
tokenizer = AutoTokenizer.from_pretrained(CKPT_DIR, use_fast=False)

# 2) 베이스 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

# 3) ✅ 핵심: tokenizer 길이로 임베딩 리사이즈 (51200 -> 51201)
base_model.resize_token_embeddings(len(tokenizer))

# (안전) pad/eos 명시
base_model.config.pad_token_id = tokenizer.pad_token_id
if tokenizer.eos_token_id is not None:
    base_model.config.eos_token_id = tokenizer.eos_token_id

# 4) LoRA 어댑터 로드해서 합치기
model = PeftModel.from_pretrained(base_model, CKPT_DIR)
model.to(device)
model.eval()

print("len(tokenizer) =", len(tokenizer))
print("emb size       =", model.get_input_embeddings().weight.shape[0])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1-1008601399.py", line 1, in <cell line: 0>
    import torch
  File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 405, in <module>
    from torch._C import *  # noqa: F403
    ^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 216, in _lock_unlock_module
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call l

TypeError: object of type 'NoneType' has no len()

10. 프롬프트 3종 비교(단순/조건/강한 제약)

In [ ]:
sample_review_neg = "배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다."
sample_review_pos = "배송도 빠르고 제품 품질이 좋아요. 재구매 의사 있습니다!"

def prompt_basic(review):
    return f"리뷰: {review}\n판매자 답변:"

def prompt_with_rules(review, neg=True):
    rules = (
        "규칙: 한국어 존댓말, 2~4문장.\n"
        + ("사과+해결책+보상(쿠폰/교환/환불 중 하나)을 반드시 포함.\n" if neg else "감사+재구매 유도 포함.\n")
    )
    return f"{rules}리뷰: {review}\n판매자 답변:"

def prompt_project_style(review):
    # 학습 때 쓰던 포맷(일관성 최고)
    return (
        "### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n"
        "### 규칙:\n"
        "- 한국어 존댓말로 정중하게 작성\n"
        "- 2~4문장으로 간결하게\n"
        "- 부정 리뷰에는 사과+해결책+보상 제안 포함\n"
        "- 긍정 리뷰에는 감사+재구매 유도 포함\n"
        "### 고객 리뷰:\n"
        f"{review}\n"
        "### 판매자 답변:\n"
    )

for p in [prompt_basic(sample_review_neg),
          prompt_with_rules(sample_review_neg, neg=True),
          prompt_project_style(sample_review_neg)]:
    print("PROMPT:\n", p)
    print("OUTPUT:\n", generate_reply(p))
    print("="*80)

for p in [prompt_basic(sample_review_pos),
          prompt_with_rules(sample_review_pos, neg=False),
          prompt_project_style(sample_review_pos)]:
    print("PROMPT:\n", p)
    print("OUTPUT:\n", generate_reply(p))
    print("="*80)


11. 규칙 준수 점수(간단 룰 기반)

In [ ]:
import re

def score_reply(review_rating, reply):
    score = 0
    # 문장 수(대충 .?! 기준)
    sentences = [s for s in re.split(r"[.?!]\s*", reply) if s.strip()]
    if 2 <= len(sentences) <= 4:
        score += 1

    if review_rating <= 2:
        if any(k in reply for k in ["죄송", "사과"]):
            score += 1
        if any(k in reply for k in ["교환", "환불", "쿠폰", "보상"]):
            score += 1
    if review_rating >= 4:
        if any(k in reply for k in ["감사", "고맙"]):
            score += 1
        if any(k in reply for k in ["재구매", "다음에도", "또 이용"]):
            score += 1
    return score

# 샘플 50개로 점수 보기
samples = random.sample(filtered, 50)
total = 0
for r, review in samples:
    p = prompt_project_style(review)
    out = generate_reply(p)
    total += score_reply(r, out)

print("avg rule score:", total/50)
